In [ ]:
# see link
# https://github.com/Fraud-Detection-Handbook

package ='07-sklearn.ensemble'
name='Bagging'
tuningAndParameters='04-Random Oversampling'

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import datamanagement as dm
reload(dm)

import result as resultMd
reload(resultMd)

import graph as gf
reload(gf)

print('done')

In [ ]:
dfLearning, dfValidation =dm.getDataLearningAndValidation()

dfLearning.head()

In [ ]:
%%script false
# found rate = 4.0

from sklearn.ensemble import BaggingClassifier
import matplotlib.pyplot as plt
from datetime import datetime
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
import numpy as np
from sklearn.metrics import f1_score

train_f1s=[]
test_f1s =[]
range= []

TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0
predictors = dm.getPredictors(dfLearning)
target = dm.getTarget()

x_train0, x_test, y_train0, y_test = train_test_split(dfLearning[predictors], dfLearning[target], test_size = TEST_SIZE, 
                                                        stratify=dfLearning[target],
                                                        random_state = RANDOM_STATE)

ratio=dfLearning[target].value_counts()[1]/dfLearning[target].value_counts()[0]
print(ratio)

rates = np.arange(1.25,12,0.25)
for rate in rates:
    print("",rate,1/rate)
    oversample = RandomOverSampler(sampling_strategy=rate*ratio,random_state=42)
    x_train, y_train = oversample.fit_resample(x_train0, y_train0)


    print(x_train0.shape)
    print(x_train.shape)
    
    modelClf = BaggingClassifier(random_state=42)
    parameters={ 'n_estimators': 75}
    modelClf.set_params(**parameters)

    modelClf.fit(x_train, y_train)
    predsTrain = modelClf.predict(x_train)
    predsTest = modelClf.predict(x_test)

    train_f1=f1_score(y_train, predsTrain)
    print("f1 train {:.4f}".format(train_f1))
    
    test_f1=f1_score(y_test, predsTest)
    print("f1 test  {:.4f}".format(test_f1))
    
    train_f1s.append(train_f1)
    test_f1s.append(test_f1)
    range.append(rate)
    print('-----------------------')

dm.plt_train_test(range, train_f1s, "train", test_f1s," test")

In [ ]:
%%script false
dm.plt_train_test(range, train_f1s, "train", test_f1s," test")

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0

predictors = dm.getPredictors(dfLearning)
target = dm.getTarget()

x_train0, x_test, y_train0, y_test = train_test_split(dfLearning[predictors], dfLearning[target], test_size = TEST_SIZE, 
                                                        stratify=dfLearning[target],
                                                        random_state = RANDOM_STATE)

ratio=dfLearning[target].value_counts()[1]/dfLearning[target].value_counts()[0]
print(ratio)
rate = 2
oversample = RandomOverSampler(sampling_strategy=rate*ratio,random_state=42)
x_train, y_train = oversample.fit_resample(x_train0, y_train0)

In [ ]:
%%script false

from sklearn.ensemble import BaggingClassifier
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

#{'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 7, 'min_samples_split': 32
dic_param={
 'n_estimators': randint(70,180)
}
modelClf = BaggingClassifier(random_state=42)

random_search = RandomizedSearchCV(modelClf,dic_param, scoring='f1', verbose=10,cv=4).fit(x_train, y_train)
print(random_search.best_params_)
print(random_search.best_score_)

#{'n_estimators': 156}
#0.9211528221094857

In [ ]:
### %%script false

from sklearn.ensemble import BaggingClassifier
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV

dic_param={
    'n_estimators': [154,156,158]
}
modelClf = BaggingClassifier(random_state=42)

random_search = GridSearchCV(modelClf,dic_param, scoring='f1', verbose=10,cv=4).fit(x_train, y_train)
print(random_search.best_params_)
print(random_search.best_score_)

#{'n_estimators': 156}
#0.9211528221094857



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.ensemble import BaggingClassifier
from datetime import datetime

modelClf = BaggingClassifier(random_state=42)
parameters={'n_estimators': 156}

modelClf.set_params(**parameters)
then= datetime.now()
modelClf.fit(x_train, y_train)
now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result(package, name, tuningAndParameters, duration_in_s)

predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

dm.show_confusion_matrix(y_train, predsTrain,'Confusion matrix learning data')
print(f"f1 train {f1_score(y_train, predsTrain):.3f}")
dm.show_confusion_matrix(y_test, predsTest,'Confusion matrix test data')
print(f"f1 test {f1_score(y_test, predsTest):.3f}")

In [ ]:
gf.show_importance(modelClf, predictors)

In [ ]:
gf.show_prediction_graph(modelClf, x_test,y_test)

In [ ]:
predsValidation = modelClf.predict(dfValidation[predictors])
f1=f1_score(dfValidation[target], predsValidation)

dm.show_confusion_matrix(dfValidation[target], predsValidation,'Confusion matrix validation data')
print(f"f1 validation {f1:.3f}")
resultMd.update_performance_result(package, name, tuningAndParameters, f1)